In [1]:
import math
import datetime
import time
import string
import re
from glob import glob

import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
import yahoo_fin.stock_info as stock_info

from fundamental.download_usa import _postprocess_ohlcv
from utils import *

In [ ]:
all_tickers = get_all_usa_tickers()

In [2]:
ticker_name = "NVDA"
yf_ticker = yf.Ticker(ticker_name)
result = _postprocess_ohlcv(yf_ticker.history(period="5d", raise_errors=True), ticker_name)
result

2025-07-14    2025-07-15    2025-07-16    2025-07-17  \
Ticker Type                                                             
NVDA   Open    1.653700e+02  1.711900e+02  1.710600e+02  1.720200e+02   
       High    1.654900e+02  1.724000e+02  1.717500e+02  1.741600e+02   
       Low     1.620200e+02  1.692000e+02  1.689000e+02  1.708300e+02   
       Close   1.640700e+02  1.707000e+02  1.713700e+02  1.730000e+02   
       Volume  1.369758e+08  2.306274e+08  1.588315e+08  1.608411e+08   

                 2025-07-18  
Ticker Type                  
NVDA   Open    1.736400e+02  
       High    1.742500e+02  
       Low     1.712600e+02  
       Close   1.724100e+02  
       Volume  1.461076e+08

In [3]:
ticker_name = "NVDA"
yf_ticker = yf.Ticker(ticker_name)

In [4]:
yf_ticker.get_earnings_estimate()

,avg,low,high,yearAgoEps,numberOfAnalysts,growth
period,,,,,,
0q,0.99713,0.96706,1.09,0.68000,40,0.4664
+1q,1.16656,1.07000,1.40,0.81000,40,0.4402
0y,4.28706,3.96000,4.65,2.99000,45,0.4338
+1y,5.76201,4.32000,7.80,4.28706,55,0.3440


In [7]:
yf_ticker.info["lastFiscalYearEnd"]

1737849600

In [8]:
pd.Timestamp(1737849600, unit="s").date()

datetime.date(2025, 1, 26)

In [41]:
result

2025-06-23    2025-06-24    2025-06-25    2025-06-26  \
Ticker Type                                                             
AA     Open    2.805000e+01  2.885000e+01  2.895000e+01  2.858000e+01   
       High    2.875000e+01  2.918000e+01  2.897000e+01  3.030000e+01   
       Low     2.788800e+01  2.822000e+01  2.813000e+01  2.854000e+01   
       Close   2.871000e+01  2.880000e+01  2.817000e+01  2.993000e+01   
       Volume  5.277800e+06  5.249100e+06  3.769300e+06  6.456200e+06   

                 2025-06-27  
Ticker Type                  
AA     Open    2.988000e+01  
       High    2.999900e+01  
       Low     2.772000e+01  
       Close   2.847000e+01  
       Volume  9.409300e+06

In [ ]:
ohlcv = pd.read_parquet("ASSETS/ohlcv.parquet")
ohlcv_index_tz = ohlcv.loc["A"].index.tz

parquet_paths = glob("DB/**/ohlcv.parquet")
for parquet_path in parquet_paths:
    end_date = datetime.datetime.strptime(parquet_path.split(os.sep)[-2], "%y%m%d")
    start_date = end_date - datetime.timedelta(days=6)
    start_date = pd.Timestamp(start_date).tz_localize(ohlcv_index_tz)
    end_date = pd.Timestamp(end_date).tz_localize(ohlcv_index_tz)

    df = pd.read_parquet(parquet_path)
    week_ohlcv = ohlcv.loc[(slice(None), slice(start_date, end_date)), :]
    new_ohlcv = []
    for ticker in week_ohlcv.index.get_level_values(0).unique().tolist():
        ticker_ohlcv = week_ohlcv.loc[ticker]
        ticker_ohlcv = week_ohlcv[["Open", "High", "Low", "Close", "Volume"]].loc["A"].T
        ticker_ohlcv.index.name = "Type"
        ticker_ohlcv["Ticker"] = ticker
        ticker_ohlcv = ticker_ohlcv.reset_index().set_index(["Ticker", "Type"])
        ticker_ohlcv.columns = [str(col.date()) for col in ticker_ohlcv.columns]
        new_ohlcv.append(ticker_ohlcv)
    new_ohlcv = pd.concat(new_ohlcv, axis=0)
    new_ohlcv.to_parquet(parquet_path.replace("price", "ohlcv"))